In [2]:
import datumaro as dm
DUP_LABELS_MAPPING = {
        'White Fish': 'Whitefish',
        'Bull Trout': 'Bull',
        'Lan prey': 'Lamprey',
        'Lampray': 'Lamprey'
        }

In [2]:
dataset = dm.Dataset.import_from("export_combined_bear_kitwanga_preprocess_yolo", "yolo")
print("Subset candidates:", dataset.subsets().keys())

2023-10-27 00:17:31.199375: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 00:17:31.239928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 00:17:32.285759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Subset candidates: dict_keys(['train', 'valid'])


In [3]:
subset = list(dataset.subsets().keys())[1]
print("Subset:", subset)

Subset: valid


In [4]:
def get_ids(dataset: dm.Dataset, subset: str):
    ids = []
    c = 0
    for item in dataset:
        if c > 8:
            break
        if item.subset == subset:
            ids += [item.id]
        c += 1

    return ids


#ids = get_ids(dataset, subset)
print("DatasetItem ids:", ids[:4])

DatasetItem ids: []


In [14]:
def load():
    dt1 = dm.Dataset.import_from('/mnt/shiorissd4tb/masamim/export_kitwanga_yolo/', 'yolo')
    #dt1.transform('remap_labels', mapping=DUP_LABELS_MAPPING)
    dt2 = dm.Dataset.import_from('/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo/', 'yolo')
    #dt2.transform('remap_labels', mapping=DUP_LABELS_MAPPING)
    return dt1, dt2

dt_kit, dt_br = load()

In [16]:
def merge():
    merged = dm.HLOps.merge(dt_kit, dt_br, merge_policy='union')
    return merged

dt_kit_br = merge()

KeyboardInterrupt: 

In [3]:
dt_kit_br_tiny = dm.Dataset.import_from('/mnt/shiorissd4tb/masamim/export_combined_kit_bear_yolo_tiny_test', 'yolo')
dt_kit_br_tiny_split = dt_kit_br_tiny.transform("split", task='detection', splits=[('train', 0.5), ('valid', 0.5)])

2023-11-03 09:01:29.702020: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 09:01:29.757307: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 09:01:30.791854: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
dm.components.operations.compute_ann_statistics(dt_kit_br_tiny_split.get_subset('valid'))

{'images count': 13915,
 'annotations count': 11426,
 'unannotated images count': 2836,
 'unannotated images': ['/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000001',
  '/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000003',
  '/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000004',
  '/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000005',
  '/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000007',
  '/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000010',
  '/home/masam

In [5]:
next(iter(dt_kit_br_tiny_split))

DatasetItem(id='/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000000', subset='train', media=ImageFromFile(path='/home/masamim/salmon-computer-vision/utils/export_bear_creek_yolo_tiny/train/08-01-2021_01-00-09_m_salmon_camera/obj_train_data/frame_000000.jpg'), annotations=[], attributes={})

In [6]:
dt_kit_br_tiny_split.export('/mnt/shiorissd4tb/masamim/export_combined_kit_bear_yolo_tiny_split', 'yolo', save_media=True, image_ext='.jpg')

In [86]:
def fix_empty_annos():
    dt_empty = dm.Dataset.import_from('/home/masamim/salmon-computer-vision/utils/empty_proj/', 'datumaro')
    #dt_empty.transform('remap_labels', mapping=DUP_LABELS_MAPPING)
    #dt_empty.export('/home/masamim/salmon-computer-vision/utils/empty_proj_fixed', 'datumaro', save_media=True)
    return dt_empty

#fix_empty_annos()

Dataset
	size=0
	source_path=/home/masamim/salmon-computer-vision/utils/empty_proj/
	media_type=<class 'datumaro.components.media.Image'>
	annotated_items_count=0
	annotations_count=0
subsets
	infos
	categories
	label: ['Coho', 'Bull', 'Rainbow', 'Sockeye', 'Pink', 'Whitefish', 'Chinook', 'Shiner', 'Pikeminnow', 'Chum', 'Steelhead', 'Jack Chinook', 'Juvenile Salmonoid', 'Lamprey', 'Cutthroat']

In [ ]:
#visualizer = dm.Visualizer(dataset, figsize=(16, 12), alpha=0.7)
#dm.Visualizer

In [ ]:
#fig = visualizer.vis_gallery(ids[:4], subset, (None, None))

In [8]:
from ultralytics import YOLO
from ultralytics import settings

# Update a setting
settings.update({'runs_dir': '/training/runs'})

In [18]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Train the model using the 'coco128.yaml' dataset for 3 epochs
#results_train = model.train(data='export_combined_bear_kitwanga_preprocess_yolo/combined_bear_kitwanga_preprocess.yaml', 
#                            epochs=100, batch=128, device=[0, 1, 2, 3])

#!yolo detect train data=export_combined_bear_kitwanga_preprocess_yolo/combined_bear_kitwanga_preprocess.yaml pretrained=/training/runs/detect/train15/weights/best.pt model=yolov8n.yaml epochs=100 imgsz=640 device=0,1,2,3 batch=512
!yolo detect train data=/mnt/shiorissd4tb/masamim/export_combined_kit_bear_yolo_tiny_split/combined_kit_bear_tiny.yaml pretrained=/training/runs/detect/train15/weights/best.pt model=yolov8n.yaml epochs=30 imgsz=640 device=0,1,2,3 batch=512


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [11]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/usr/src/datasets',
 'weights_dir': '/usr/src/ultralytics/weights',
 'runs_dir': '/training/runs',
 'uuid': '8765e340dc3a16f2d81ba2b190673429a59e3c2aa9083be50ddaa72612f8b714',
 'sync': True,
 'api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [ ]:
# Evaluate the model's performance on the validation set
results_val = model.val()

In [ ]:
success = model.export(format='onnx', dynamic=True)

In [ ]:
# Initialize the YOLO model
model = YOLO('yolov8n.yaml')

# Tune hyperparameters for 30 epochs
model.tune(data='/mnt/shiorissd4tb/masamim/export_combined_kit_bear_yolo_tiny_split/combined_kit_bear_tiny.yaml', pretrained='/training/runs/detect/train15/weights/best.pt', epochs=30, iterations=300, batch=512, device=[0,1,2,3], plots=False, save=False, val=False)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128